In [8]:
import functools # https://docs.python.org/3/library/functools.html
import glob # short for global, is a function that's used to search for files that match a specific file pattern or name.
import os
import csv
import copy
from collections import namedtuple

import SimpleITK as sitk # https://simpleitk.org/ (We can treat the format of the data files as a black box and use SimpleITK to load them into more familiar NumPy arrays.)
import numpy as np
import torch
from torch.utils.data import Dataset

from util.util import XyzTuple, xyz2irc
from util.disk import getCache
from util.logconf import logging

In [9]:
CandidateInfoTuple = namedtuple(
    'CandidateInfoTuple',
    'isNodule_bool, diameter_mm, series_uid, center_xyz',
)

### getCandidateInfoList comments

* *@functools.lru_cache(1) - in-memory cache decorator*
* *requireOnDisk_bool=True - defaults to screening out series from data subsets that aren't in place yet.*
* *We construct a set with all series_uids that are present on disk. This will let us use the data, even if we haven't downloaded all of the subsets yet.*
* *For each of the candidate entries for a given series_uid, we loop through the annotations we collected earlier for the same series_uid and see if the two coordinates are
close enough to consider them the same nodule.*
* *If we don’t find diameter information for a nodule, that’s fine; we’ll just treat the nodule as having a 0.0 diameter.*

In [15]:
@functools.lru_cache(1)
def getCandidateInfoList(requireOnDisk_bool=True):
    mhd_list = glob.glob('data/subset*/*.mhd') # Return a possibly empty list of path names that match pathname
    present_on_disk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}
    
    print("candidates present on disk: ", present_on_disk_set)

    # First we need to group our annotations by series_uid, as that’s the first key we’ll use to cross-reference each row from the two files.
    diameter_dict = {}

    # seriesuid,coordX,coordY,coordZ,diameter_mm - header of 'annotations.csv'
    with open('data/annotations.csv', 'r') as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm = float(row[4])

            diameter_dict.setdefault(series_uid, []).append(
                (annotationCenter_xyz, annotationDiameter_mm)
            )

    # Now we’ll build our full list of candidates using the information in the candidates.csv file.
    candidateInfo_list = []

    # seriesuid,coordX,coordY,coordZ,class - header of 'candidates.csv'
    with open('data/candidates.csv', 'r') as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]

            # If a series_uid isn’t present, it’s in a subset we don’t have on disk, so we should skip it.
            if series_uid not in present_on_disk_set and requireOnDisk_bool:
                continue

            isNodule = bool(int(row[4]))
            candidateCenter_xyz = tuple([float(x) for x in row[1:4]])

            candidateDiameter_mm = 0.0
            for annotation_tuple in diameter_dict.get(series_uid, []):
                annotationCenter_xyz, annotationDiameter_mm = annotation_tuple
                for i in range(3):
                    delta_mm = abs(candidateCenter_xyz[i] - annotationCenter_xyz[i])
                    
                    # Divides the diameter by 2 to get the radius, and divides the radius by 2 to require that the two nodule center points not be too far apart relative to the size of the nodule. 
                    # (This results in a bounding-box check, not a true distance check.) 
                    if delta_mm > annotationDiameter_mm / 4:
                        break
                    else:
                        candidateDiameter_mm = annotationDiameter_mm
                        break

                candidateInfo_list.append(CandidateInfoTuple(
                    isNodule,
                    candidateDiameter_mm,
                    series_uid,
                    candidateCenter_xyz,
                ))

    # This means we have all the actual nodule samples starting with the largest first, 
    # followed by all the non-nodule samples (which don’t have nodule size information).
    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list

* *The 10 subsets we discussed earlier have about 90 CT scans each (888 in total), with every CT scan represented as two files: one with a .mhd extension and one with a .raw extension. The data being split between multiple files is hidden behind the sitk routines, however, and is not something we need to be directly concerned with.*

* *Continuing the __init__ method, we need to do a bit of cleanup on the ct_a val- ues. CT scan voxels are expressed in Hounsfield units (HU; https://en.wikipedia.org/wiki/Hounsfield_scale), which are odd units; air is –1,000 HU (close enough to 0 g/cc [grams per cubic centimeter] for our purposes), water is 0 HU (1 g/cc), and bone is at least +1,000 HU (2–3 g/cc).*

* *It’s important to know that our data uses the range of –1,000 to +1,000.*

* *Candidate center data is expressed in millimeters, not voxels. We need to transform our coordinates from the millimeter-based coordinate system (X,Y,Z) they’re expressed in, to the voxel-address-based coordinate system (I,R,C) used to take array slices from our CT scan data.*

# ![title](images/convertToIRC.png)

When dealing with CT scans, we refer to the array dimensions as index, row, and column, because a separate meaning exists for X, Y, and Z, as illustrated in figure 10.6. The patient coordinate system defines positive X to be patient- left (left), positive Y to be patient-behind (posterior), and positive Z to be toward-patient- head (superior). **Left-posterior-superior is sometimes abbreviated LPS.**

# ![title](images/figure10_6.png)

* The patient coordinate system is measured in millimeters and has an arbitrarily positioned origin that does not correspond to the origin of the CT voxel array.

# ![title](images/figure10_7.png)

When plotted using square pixels, the non-cubic voxels can end up looking some- what distorted, similar to the distortion near the north and south poles when using a Mercator projection map. We will need to apply a scaling factor if we want the images to depict realistic proportions.

* CTs are commonly 512 rows by 512 columns, with the index dimension ranging from around 100 total slices up to perhaps 250 slices (250 slices times 2.5 millimeters is typically enough to contain the anatomical region of interest). This results in a lower bound of approximately 225 voxels, or about 32 million data points. Each CT specifies the voxel size in millimeters as part of the file metadata;

Do the math manually:
* Flip the coordinates from IRC to CRI, to align with XYZ.
* Scale the indices with the voxel sizes.
* Matrix-multiply with the directions matrix, using @ in Python.
* Add the offset for the origin.

To go back from XYZ to IRC, we need to perform the inverse of each step in the reverse order. The metadata we need to convert from patient coordinates (_xyz) to array coordinates (_irc) is contained in the MetaIO file alongside the CT data itself. We pull the voxel sizing and positioning metadata out of the .mhd file at the same time we get the ct_a.

* The getRawNodule function takes the center expressed in the patient coordinate sys- tem (X,Y,Z), just as it’s specified in the LUNA CSV data, as well as a width in voxels. It returns a cubic chunk of CT, as well as the center of the candidate converted to array coordinates.

In [11]:
class CT:
    def __init__(self, series_uid):
        mhd_path = glob.glob(
            'data/subset*/{}.mhd'.format(series_uid) # We don’t care to track which subset a given series_uid is in, so we wildcard the subset.
        )[0]

        # sitk.ReadImage implicitly consumes the .raw file in addition to the passed-in .mhd file.
        ct_mhd = sitk.ReadImage(mhd_path)

        # Recreates an np.array since we want to convert the value type to np.float32
        # ct_a is a three-dimensional array.
        ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

        # Capping the values between -1000 and 1000
        ct_a.clip(-1000, 1000, ct_a)

        self.series_uid = series_uid
        self.hu_a = ct_a

        self.origin_xyz = XyzTuple(*ct_mhd.GetOrigin())
        self.vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())
        self.direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)  # Converts the directions to an array, and reshapes the nine-element array to its proper 3x3 matrix shape

    def getRawCandidate(self, center_xyz, width_irc):
        center_irc = xyz2irc(
            center_xyz,
            self.origin_xyz,
            self.vxSize_xyz,
            self.direction_a
        )

        slice_list = []
        for axis, center_val in enumerate(center_irc):
            start_ndx = int(round(center_val - width_irc[axis] / 2))
            end_ndx = int(start_ndx + width_irc[axis])

            assert center_val >= 0 and center_val < self.hu_a.shape[axis], repr([self.series_uid, center_xyz, self.origin_xyz, self.vxSize_xyz, center_irc, axis])

            if start_ndx < 0:
                # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
                #     self.series_uid, center_xyz, center_irc, self.hu_a.shape, width_irc))
                start_ndx = 0
                end_ndx = int(width_irc[axis])

            if end_ndx > self.hu_a.shape[axis]:
                # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
                #     self.series_uid, center_xyz, center_irc, self.hu_a.shape, width_irc))
                end_ndx = self.hu_a.shape[axis]
                start_ndx = int(self.hu_a.shape[axis] - width_irc[axis])

            slice_list.append(slice(start_ndx, end_ndx))

        ct_chunk = self.hu_a[tuple(slice_list)]

        return ct_chunk, center_irc

# ![title](images/figure10_8.png)

* By subclassing Dataset, we will take our arbitrary data and plug it into the rest of the PyTorch ecosystem.

* Our LunaDataset class will normalize those samples, flattening each CT’s nodules into a single collection from which samples can be retrieved without regard for which Ct instance the sample originates from.

The PyTorch API only requires that any Dataset subclasses we want to implement must provide these two functions:
* An implementation of __len__ that must return a single, constant value after initialization (the value ends up being cached in some use cases)
* The __getitem__ method, which takes an index and returns a tuple with sample data to be used for training (or validation, as the case may be)

In [12]:
raw_cache = getCache('part2ch10_raw')

@functools.lru_cache(1, typed=True)
def getCt(series_uid):
    return CT(series_uid)

# The implementation of 'raw_cache' in util folder
@raw_cache.memoize(typed=True)
def getCtRawCandidate(series_uid, center_xyz, width_irc):
    ct = getCt(series_uid)
    ct_chunk, center_irc = ct.getRawCandidate(center_xyz, width_irc)
    return ct_chunk, center_irc

In [13]:
log = logging.getLogger(__name__)
# log.setLevel(logging.WARN)
# log.setLevel(logging.INFO)
log.setLevel(logging.DEBUG)

In [14]:
class LunaDataset(Dataset):
    def __init__(self,
                 val_stride=0,
                 isValSet_bool=None,
                 series_uid=None,
                 ):
        self.candidateInfo_list = copy.copy(getCandidateInfoList()) # Copies the return value so the cached copy won’t be impacted by altering self.candidateInfo_list

        if series_uid:
            self.candidateInfo_list = [
                x for x in self.candidateInfo_list if x.series_uid == series_uid
            ]

        if isValSet_bool:
            assert val_stride > 0, val_stride
            self.candidateInfo_list = self.candidateInfo_list[::val_stride]
            assert self.candidateInfo_list
        elif val_stride > 0:
            del self.candidateInfo_list[::val_stride] # Deletes the validation images (every val_stride-th item in the list) from self.candidateInfo_list. We made a copy earlier so that we don’t alter the original list.
            assert self.candidateInfo_list

        log.info("{!r}: {} {} samples".format(
            self,
            len(self.candidateInfo_list),
            "validation" if isValSet_bool else "training",
        ))

    def __len__(self):
        return len(self.candidateInfo_list)
    
    def __getitem__(self, ndx):
        candidateInfo_tup = self.candidateInfo_list[ndx]
        width_irc = (32, 48, 48)

        candidate_a, center_irc = getCtRawCandidate(    # The return value candidate_a has shape (32,48,48), the axes are depth, height, and width
            candidateInfo_tup.series_uid,
            candidateInfo_tup.center_xyz,
            width_irc
        )

        # The next thing we need to do in the __getitem__ method is manipulate the data into the proper data types and required array dimensions that will be expected by downstream code.
        candidate_t = torch.from_numpy(candidate_a)
        candidate_t = candidate_t.to(torch.float32)
        candidate_t = candidate_t.unsqueeze(0) # .unsqueeze(0) adds the 'Channel' dimension

        # This has two elements, one each for our possible candidate classes (nodule or non- nodule; or positive or negative, respectively). 
        pos_t = torch.tensor([
                not candidateInfo_tup.isNodule_bool,
                candidateInfo_tup.isNodule_bool
            ],
            dtype=torch.long
        )

        return (
            candidate_t,
            pos_t,
            candidateInfo_tup.series_uid,
            torch.tensor(center_irc)
        )